# An Example: A Linear-Gaussian Latent Feature Model with Binary Features



## Finite Linear-Gaussian Model

$\mathbf{x}_i$ is generated from a Gaussian with:

- mean $\mathbf{z}_i \mathbf{A}$
- covariance $\Sigma_X = \sigma^2_X \mathbf{I}$

Where:

- $\mathbf{z}_i$ is a $K$-dimensional binary vector
- $\mathbf{A}$ is a $K$ x $D$ matrix of weights

Since the mean of $\mathbf{x}_i$ is $\mathbf{z}_i \mathbf{A}$, so the expectation of $\mathbf{X}$, $\mathbb{E}(\mathbf{X})$, is $\mathbf{Z} \mathbf{A}$.

Given that:

- $\mathbf{X}$ is an $N$ x $D$ matrix

So:

- $\mathbf{Z}$ is an $N$ x $K$ matrix

For the sake of thinking this through, this means that for example each observation $\mathbf{x}_i$ could be low-dimensional, eg 1 or 2 dimensions, whilst the latent feature matrix $\mathbf{Z}$ could have high-dimensional $K$, eg 1000, or more.

Note that the covariance matrix is diagonal, so the Gaussians have spherical isocontours.  This constrains the solution, simplifying inference.

As per the tutorial, and since each $\mathbf{x}_i$ distributed as a symmetric Gaussian, the distribution of $\mathbf{X}$, given $\mathbf{Z}$, $\mathbf{A}$ and $\sigma_X$ is:

$$
p(\mathbf{X} \mid \mathbf{Z}, \mathbf{A}, \sigma_X)
=
\frac
  {1}
  {(2\pi \sigma_X^2)^{ND/2}}
\exp \left(
  -
  \frac{1}
    {2\sigma_X^2}
    \mathrm{tr}
    (
      (\mathbf{X} - \mathbf{Z}\mathbf{A})^T
      (\mathbf{X} - \mathbf{Z}\mathbf{A})
    )
\right)
$$


Note that since $\mathbf{Z}$ is a dimension in $N$, so the mean of each $\mathbf{x}_i$ can be different.  $\mathbf{A}$ does not have a dimension in $N$, and gives the properties of each mixing component. (I think).

The tutorial suggests that we should integrate out the model components $\mathbf{A}$, and that we can do so, if we define a prior on it, which the tutorial suggests to use a matrix Gaussian:

$$
p(\mathbf{A} \mid \sigma_A) =
\frac{1}
  {(2 \pi \sigma_A^2)^{KD/2}}
\exp \left(
  - \frac{1}{2\sigma_A^2}
  \mathrm{tr}(\mathbf{A}^T \mathbf{A})
\right)
$$

Multiplying these two probabilities we get the joint probability of $\mathbf{X}$ and $\mathbf{A}$:

$$
p(\mathbf{X}, \mathbf{A} \mid \mathbf{Z}, \sigma_X)
= p(\mathbf{X} \mid \mathbf{Z}, \mathbf{A}, \sigma_X) \, p(\mathbf{A} \mid \sigma_A)
$$
$$
= 
\frac
  {1}
  {(2\pi \sigma_X^2)^{ND/2}}
\exp \left(
  -
  \frac{1}
    {2\sigma_X^2}
    \mathrm{tr}
    (
      (\mathbf{X} - \mathbf{Z}\mathbf{A})^T
      (\mathbf{X} - \mathbf{Z}\mathbf{A})
    )
\right)
\cdot
\frac{1}
  {(2 \pi \sigma_A^2)^{KD/2}}
\exp \left(
  - \frac{1}{2\sigma_A^2}
  \mathrm{tr}(\mathbf{A}^T \mathbf{A})
\right)
$$

Gradually working through the tutorial expressions:

$$
\propto
\exp
\left(
- \frac{1}{2}
  \mathrm{tr} \left(
    \frac{1}{\sigma_X^2}\mathbf{X}^T\mathbf{X}
    - \frac{1}{\sigma_X^2} \mathbf{A}^T\mathbf{Z}^T\mathbf{X}
    - \frac{1}{\sigma_X^2} \mathbf{X}^T \mathbf{Z} \mathbf{A}
    + \frac{1}{\sigma_X^2} \mathbf{A}^T\mathbf{Z}^T\mathbf{Z}\mathbf{A}
    + \frac{1}{\sigma_A^2} \mathbf{A}^T \mathbf{A}
  \right)
\right)
$$

$$
= \exp \left(
  - \frac{1}{2}
  \mathrm{tr} \left(
    \frac{1}{\sigma_X^2}\mathbf{X}^T\mathbf{X}
    - \frac{1}{\sigma_X^2} \mathbf{A}^T\mathbf{Z}^T\mathbf{X}
    - \frac{1}{\sigma_X^2} \mathbf{X}^T \mathbf{Z} \mathbf{A}
    + \mathbf{A}^T(\frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z} + \frac{1}{\sigma_A^2}\mathbf{I})\mathbf{A}
  \right)
\right)
$$

## Interlude: Matrix and Gaussian revision

At this point, I had to reach out to revise some properties of matrices and Gaussians that I ~~had forgotten~~ never learned.  Some of the resources I used for this section:

- "Bayesian Linear Regression", Minka, 1998 (revised 2010)
- https://en.wikipedia.org/wiki/Gaussian_integral
- https://en.wikipedia.org/wiki/Multivariate_normal_distribution
- https://en.wikipedia.org/wiki/Matrix_normal_distribution
- [https://en.wikipedia.org/wiki/Vectorization_(mathematics)](https://en.wikipedia.org/wiki/Vectorization_(mathematics%29)
- https://en.wikipedia.org/wiki/Conjugate_transpose

(Note that it turns out in hind-sight, that it is sufficient to remember that probability distributions integrate to 1, to know how to integrate expressions in the form of a matrix-normal distribution, so you can skip pretty much most of the below section if you want.  But personally I thought learning about Kronecker products and matrix-normal distributions was really interesting, and gave me a bit more confidence that my maths is fractionally less terrible than it is)

### Matrix normal distribution, first glance

So, going through, bit by bit: the integral of a Multivariate normal distribution is:

$$
\int_{-\infty}^{\infty}
\exp \left(
   -\frac{1}{2}
   (\mathbf{x}^T \mathbf{A} \mathbf{x})
\right)
\,
d\mathbf{x}
=
\sqrt{\frac{(2\pi)^D}{\mathrm{det}\,\mathbf{A}}}
$$

*However*, in this case, $\mathbf{x}$ is a vector, but we will have $\mathbf{X}$, or something of this sort, ie: a matrix.  So, what we will have is in fact: a Matrix normal distribution.  A Matrix normal distribution, per wikipedia article, has the form:

$$
p(\mathbf{X} \mid \mathbf{M}, \mathbf{U}, \mathbf{V})
= \frac
  {\exp(
     -\frac{1}{2}
     \mathrm{tr}(
       \mathbf{V}^{-1}
       (\mathbf{X} - \mathbf{M})^T
       \mathbf{U}^{-1}
       (\mathbf{X} - \mathbf{M})
     )
  }
  {(2\pi)^{NK/2} \left| \mathbf{V} \right|^{N/2} \left| \mathbf{U} \right|^{K/2}}
$$

where:

- $\mathrm{tr}$ denotes "trace"
- $\mathrm{M}$ is $N$ x $K$
- $\mathrm{U}$ is $N$ x $N$ (so: square)
- $\mathrm{V}$ is $v$ x $v$ (also square)

(where I've changed $n$ in the Wikipedia article to $N$, and $p$ to $K$, in line with the notation we are using elsewhere)

The article then states that the relationship to Multivariate normal is:

$$
\mathrm{vec}(\mathbf{X})
\sim
\mathcal{N}_{NK}(\mathrm{vec}(\mathbf{M}, \mathbf{V} \otimes \mathbf{U}))
$$

where:

- $\otimes$ is Kronecker Product
- $\mathrm{vec}$ is vectorization

### Vectorization

What is Kronecker Product, and what is vectorization?  The [Wikipedia article](https://en.wikipedia.org/wiki/Vectorization_(mathematics%29) gives a good description of vectorization.  You stack each column of the matrix on top of each other, so it becomes a vector:

If:

$$
\mathbf{A}
=
\begin{bmatrix} a & b \\
c & d \\
\end{bmatrix}
$$

Then:

$$
\mathrm{vec}(\mathbf{A})
=
\begin{bmatrix}
a \\
c \\
b \\
e \\
\end{bmatrix}
$$

So, if $\mathbf{A}$ is $M$ x $N$, and the result of $\mathrm{vec}(\mathbf{A})$ is $C$, then:

$$
a_{i,j}
= 
c_{jM + i,1}
$$


### Kronecker Product

The Kronecker product of matrices $\mathbf{A}$ and $\mathbf{B}$ is formed by tiling the matrices $\mathbf{A}$ and $\mathbf{B}$ as follows, and then forming the per-element product.  If we have the following matrices:

$$
\mathbf{A}
=
\begin{bmatrix} a & b \\
c & d \\
\end{bmatrix}
$$

$$
\mathbf{B}
=
\begin{bmatrix} e & f \\
g & h \\
\end{bmatrix}
$$

Then matrix $\mathbf{A}$ will be tiled like:

$$
\begin{bmatrix}
a & a & b & b \\
a & a & b & b \\
c & c & d & d \\
c & c & d & d \\
\end{bmatrix}
$$

Matrix $\mathbf{B}$ will be tiled like:

$$
\begin{bmatrix}
e & f & e & f \\
g & h & g & h \\
e & f & e & f \\
g & h & g & h \\
\end{bmatrix}
$$

... and the Kronecker product is the Hadamard (per-element) product of these tiled matrices:

$$
\begin{bmatrix}
ae & af & be & bf \\
ag & ah & bg & bh \\
ce & cf & de & df \\
cg & ch & dg & dh \\
\end{bmatrix}
$$



Let's compare this with the matrix product of these two matrices.  This is:

$$
\begin{bmatrix}
ae + bg & af + bh \\
ce + dg & cf + dh \\
\end{bmatrix}
$$

There are 8 pairs of products in the matrix product above, and 16 in this Kronecker product, so it seems not obvious to relate the two, eg via vectorization, somehow?

### Relationship between vectorization and inner product

Wikipedia states that we can form a relationship between vectorization and the matrix product. For square, real matrices:

$$
\mathrm{tr}
(\mathbf{A}^T \mathbf{B})
= \mathrm{vec}(\mathbf{A})^T \mathrm{vec}(\mathbf{B})
$$


Let's try this, for the example matrices above.

$\mathrm{vec}(\mathbf{A})$ is:

$$
\begin{bmatrix}
a \\
c \\
b \\
d \\
\end{bmatrix}
$$

$\mathrm{vec}(\mathbf{B})$ is:

\begin{bmatrix}
e \\
g \\
f \\
h \\
\end{bmatrix}



So:

$$\mathrm{vec}^T(\mathbf{A})\mathrm{vec}(\mathbf{B}) = ae + cg + bf + dh$$


$\mathbf{A}^T$ is:
$$
\begin{bmatrix} a & c \\
b & d \\
\end{bmatrix}
$$

...and $\mathbf{B}$ is:

$$
\begin{bmatrix} e & f \\
g & h \\
\end{bmatrix}
$$


So, $\mathbf{A}^T \mathbf{B}$ is:

$$
\begin{bmatrix}
ae + cg & af + ch \\
be + dg & bf + dh \\
\end{bmatrix}
$$

$\mathrm{tr}$ is the sum of the diagonal, for a square matrix.  For example, for matrix $\mathrm{X}$ of size $m$, $\mathrm{tr}(\mathbf{X})$ is:

$$
\mathrm{tr}(\mathbf{X}) = \sum_{i=1}^m x_{i,i}
$$

So, $\mathrm{tr}(\mathbf{A}^T \mathbf{B})$, in our example above, is:

$$
\mathrm{tr}(\mathbf{A}^T \mathbf{B})
= ae + cg + bf + dh
$$

... which matches the result for $\mathrm{vec}(\mathbf{A})^T \mathrm{vec}(\mathbf{B})$

More generally, let's try for two $n$ x $n$ matrices $\mathbf{A}$ and $\mathbf{B}$.  The vectorizations will look like:

$$
a_{i,j} = \mathrm{vec}(\mathbf{A})_{jn + i}
$$

$$
b_{k,l} = \mathrm{vec}(\mathbf{B})_{ln + k}
$$

To form the inner product of the vectorizations, let's use two nested sums.  The innermost sum will be over each row in a column, and the outermost will be over columns.  So this will give:

$$
\sum_{i=1}^{n} 
\sum_{j=1}^{n}
a_{j,i} b_{j,i}
$$

Meanwhile, $\mathbf{A}^T$ is:

$$
(\mathbf{A}^T)_{i,j} = a_{j,i}
$$

... and the matrix product $\mathbf{A}^T\mathbf{B}$ is:

$$
(\mathbf{A}^T\mathbf{B})_{i,j} = \sum_{k=1}^n a_{k,i} b_{k,j}
$$

The trace of this, is the sum over the diagonal, ie the sum of terms where $i = j$.  This gives:

$$
\mathrm{tr}(\mathbf{A}^T\mathbf{B}) = \sum_{l=1}^n \sum_{k=1}^n a_{k,l} b_{k,l}
$$

By inspection, this is identical to the expression for $\mathrm{vec}(\mathbf{A})^T \mathrm{vec}(\mathbf{B})$

### Relationship between vectorization and Kronecker product

Wikipedia asserts that:

$$
\mathrm{vec}(\mathbf{A} \mathbf{B})
= (\mathbf{I}_m \otimes \mathbf{A})\mathrm{vec}(\mathbf{B})
$$

Let's try for the example matrices above:

$\mathbf{I}_m \otimes \mathbf{A}$ is Hadamard product of:

$$
\begin{bmatrix}
1 & 1 & 0 & 0 \\
1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1 \\
0 & 0 & 1 & 1 \\
\end{bmatrix}
$$

and:

$$
\begin{bmatrix}
a & b & a & b \\
c & d & c & d \\
a & b & a & b \\
c & d & c & d \\
\end{bmatrix}
$$

which is:

$$
\begin{bmatrix}
a & b & 0 & 0 \\
c & d & 0 & 0 \\
0 & 0 & a & b \\
0 & 0 & c & d \\
\end{bmatrix}
$$

$\mathrm{vec}(\mathbf{B})$ is:

$$
\begin{bmatrix}
e \\
g \\
f \\
h \\
\end{bmatrix}
$$


So, $(\mathbf{I}_m \otimes \mathbf{A})\mathrm{vec}(\mathbf{B})$ is:

$$
\begin{bmatrix}
ae + bg \\
ce + dg \\
af + bh \\
cf + dh \\
\end{bmatrix}
$$

Meanwhile, $\mathbf{A} \mathbf{B}$ is:

$$
\begin{bmatrix}
ae + bg & af + bh \\
ce + dg & cf + dh \\
\end{bmatrix}
$$

And vectorization of this is:

$$
\begin{bmatrix}
ae + bg \\
ce + dg \\
af + bh \\
cf + dh \\
\end{bmatrix}
$$

... which matches the value in this case for $(\mathbf{I}_m \otimes \mathbf{A})\mathrm{vec}(\mathbf{B})$

### Relationship between Kronecker product and vectorization, part 2

More generally, we have:

$$
(\mathbf{B}^T \otimes \mathbf{A}) \mathrm{vec}(\mathbf{X})
= \mathrm{vec}(\mathbf{A} \mathbf{X} \mathbf{B})
$$

Let's try this for the example matrices $\mathbf{A}$ and $\mathbf{B}$ above, and a new example matrix:

$$
\mathbf{X} = \begin{bmatrix}
i & j \\
k & l \\
\end{bmatrix}
$$

So, $\mathbf{B}^T$ is:

$$
\begin{bmatrix}
e & g \\
f & h \\
\end{bmatrix}
$$

$\mathbf{B}^T \otimes \mathbf{A}$ is Hadamard product, $\odot$, of:

$$
\begin{bmatrix}
e & e & g & g \\
e & e & g & g \\
f & f & h & h \\
f & f & h & h \\
\end{bmatrix}
$$

and

$$
\begin{bmatrix}
a & b & a & b\\
c & d & c & d \\
a & b & a & b\\
c & d & c & d \\
\end{bmatrix}
$$

which is:

$$
\begin{bmatrix}
ea & eb & ga & gb\\
ec & ed & gc & gd \\
fa & fb & ha & hb\\
fc & fd & hc & hd \\
\end{bmatrix}
$$

$\mathrm{vec}(\mathbf{X})$ is:

$$
\begin{bmatrix}
i \\
k \\
j \\
l \\
\end{bmatrix}
$$

And so $(\mathbf{B}^T \otimes \mathbf{A}) \mathrm{vec}(\mathbf{X})$ is:

$$
\begin{bmatrix}
eai + ebk + gaj + gbl \\
eci + edk + gcj + gdl \\
fai + fbk + haj + hbl \\
fci + fdk + hcj + hdl \\
\end{bmatrix}
$$

Let's now calculate $\mathrm{vec}(\mathbf{A}\mathbf{X}\mathbf{B})$:

$\mathbf{A}\mathbf{X}$ is:

$$
\begin{bmatrix}
ai + bk & aj + bl \\
ci + dk & cj + dl \\
\end{bmatrix}
$$

So, $\mathbf{A}\mathbf{X}\mathbf{B}$ is:

\begin{bmatrix}
aie + bke + ajg + blg & aif + bkf + ajh + blh \\
cie + dke + cjg + dlg & cif + dkf + cjh + dlh \\
\end{bmatrix}

And then vectorization of this is:

\begin{bmatrix}
aie + bke + ajg + blg \\
cie + dke + cjg + dlg \\
aif + bkf + ajh + blh \\
cif + dkf + cjh + dlh \\
\end{bmatrix}

... which matches the result for $(\mathbf{B}^T \otimes \mathbf{A})\mathrm{vec}(\mathbf{X})$ above.

Note also that this means that:

$$\mathrm{vec}(AB) = (\mathbf{B}^T \otimes \mathbf{A})\mathrm{vec}(\mathbf{I})$$

Let's try, with the example matrices above.  $\mathbf{B}^T \otimes \mathbf{A}$ is:

$$
\begin{bmatrix}
ea & eb & ga & gb\\
ec & ed & gc & gd \\
fa & fb & ha & hb\\
fc & fd & hc & hd \\
\end{bmatrix}
$$

$\mathrm{vec}(\mathbf{I})$ is:

$$
\begin{bmatrix}
1 \\
0 \\
0 \\
1 \\
\end{bmatrix}
$$

So, forming the matrix product of these we have:

$$
\begin{bmatrix}
ea + gb \\
ec + gd \\
fa + hb \\
fc + hd \\
\end{bmatrix}
$$

$\mathbf{A}\mathbf{B}$ is:

$$
\begin{bmatrix}
ae + bg & af + bh \\
ce + dg & cf + dh \\
\end{bmatrix}
$$

... and the vectorization of this matches $(\mathbf{B}^T \otimes \mathbf{A}) \mathrm{vec}(\mathbf{I})$

### Kronecker product, intuition

The Kronecker product forms every possible pair of products between the elements of the two input matrices.  It doesnt add any of these terms though, unlike a matrix product.  To form a matrix product, we therefore need some way of:

- filtering out the terms we want (eg for two 2x2 matrices, the Kronecker product has 16 pairs of products, but we only need 8 for the matrix product)
- adding these

By forming the matrix product with the vectorization of the identity matrix $\mathbf{I}$, we can handle both of these requirements.

But the Kronecker product is more general than this, since it contains every pair of products between the two input matrices.  Hence eg can be used to form the matrix product of three matrices, $\mathbf{A}\mathbf{X}\mathbf{B}$, using only one single matrix product.  (The Kronecker product itself is a tiling operation followed by per-element multiply).

### Matrix-normal distribution revisited

Working through the Minka paper, "Bayesian linear regression", and starting at section 1, we have the following data model:

$$\mathbf{y} = \mathbf{A} \mathbf{x} + \mathbf{e}$$

$$\mathbf{e} \sim \mathcal{N}(0, \mathbf{V})$$

$$p(\mathbf{y} \mid \mathbf{x}, \mathbf{A}, \mathbf{V}) \sim \mathcal{N}(\mathbf{A}\mathbf{x}, \mathbf{V})$$

where:

- $\mathbf{x}$ is an input vector of length $m$, ie $m$ features
- $\mathbf{A}$ is a coefficient matrix, or personally I'd normally call this the "parameter", or "weight", matrix
- $\mathbf{e}$ is Gaussian noise
- $\mathbf{y}$ is the output vector, of length $d$

We are only concerned with the conditional model for $\mathbf{y}$, $p(\mathbf{y} \mid \mathbf{x})$.  The distribution of $\mathbf{x}$ is not considered, and is irrelevant for the contents of Minka's paper.

We have $N$ inpt vectors, $\mathbf{x}_1,\dots,\mathbf{x}_N$ and target values $\mathbf{y}_1,\dots,\mathbf{y}_N$ forming a data set of $N$ exchangeable data points $D = \{(\mathbf{y}_1,\mathbf{x}_1),\dots,(\mathbf{y}_N,\mathbf{x}_N)\}$.  $\mathbf{Y}$ is $[\mathbf{y}_1 \dots \mathbf{y}_N]$, and $\mathbf{X}$ is $[\mathbf{x}_1 \dots \mathbf{x}_N]$.  The distribution of $\mathbf{Y}$ given $\mathbf{X}$ is:

$$p(\mathbf{Y} \mid \mathbf{X}, \mathbf{A}, \mathbf{V})
= \prod_{i=1}^N
p(\mathbf{y}_i \mid \mathbf{x}_i, \mathbf{A}, \mathbf{V})
$$


$$
=\prod_{i=1}^N
\frac{1}{\left| 2\pi\mathbf{V}\right|^{1/2}}
\exp \left(
  -\frac{1}{2}
     (\mathbf{y}_i - \mathbf{A}\mathbf{x}_i)^T\mathbf{V}^{-1}(\mathbf{y}_i - \mathbf{A}\mathbf{x}_i)
\right)
$$

$$
=
\frac{1}{\left| 2\pi\mathbf{V}\right|^{N/2}}
\exp \left(
   - \frac{1}{2}
   \sum_{i=1}^N(\mathbf{y}_i - \mathbf{A}\mathbf{x}_i)^T \mathbf{V}^{-1}(\mathbf{y}_i - \mathbf{A}\mathbf{x}_i)
\right)
$$


Minka then writes down the sum inside the exponential as a trace of matrix products:

$$
\mathrm{tr}(
  \mathbf{V}^{-1}(\mathbf{Y} - \mathbf{A}\mathbf{X})(\mathbf{Y} - \mathbf{A}\mathbf{X})^T)
)
$$

I can kind of see this is plausibly the same, but I think it might be educational to work through it.

The trace of a square matrix $\mathbf{X}$ is:

$$
\mathrm{tr}(\mathbf{X})
= \sum_{i=1}^m x_{i,i}
$$

where $m$ is the size of each dimension of the square matrix $\mathbf{X}$

Let's say we have:

$$
\mathbf{x}_i^T \mathbf{R} \mathbf{x}_i
$$

where $\mathbf{x}_i$ is a vector, and $\mathbf{R}$ is a matrix.

And form the sum over $N$ values of $\mathbf{x}_i$:

$$
\sum_{i=1}^N \mathbf{x}_i^T \mathbf{R}\mathbf{x}_i
$$

$$
(\mathbf{x}_i^T \mathbf{R})_{j}
=
\sum_{k=1}^m x_{i,k} r_{k,j}
$$

$$
(\mathbf{x}_i^T \mathbf{R}\mathbf{x}_i)
=
\sum_{l=1}^m
\sum_{k=1}^m
x_{i,k} r_{k,l} x_{i,l}
$$

(where here, $x_{i,j}$ means: the $j$th value of the vector $\mathbf{x}_i$)

Meanwhile, let's form the product $\mathbf{X}^T \mathbf{R} \mathbf{X}$, and investigate the properties of this product:

$$
(\mathbf{X}^T \mathbf{R})_{i,j}
=
\sum_{k=1}^m
x_{k,i}
r_{k,j}
$$

$$
(\mathbf{X}^T \mathbf{R} \mathbf{X})_{i,j}
= \sum_{l=1}^m
\sum_{k=1}^m
x_{k,i}
r_{k,l}
x_{l,j}
$$

Let's obtain each component $j$ of the $\mathrm{diag}$ of this result:

$$
\mathrm{diag}(
(\mathbf{X}^T \mathbf{R} \mathbf{X}))_{j}
= \sum_{l=1}^m
\sum_{k=1}^m
x_{k,j}
x_{l,j}
r_{k,l}
$$

By inspection, each component of $j$ of the diagonal corresponds to each value of $\mathbf{x}_i^T\mathbf{R}\mathbf{x}_i$, for $i = j$, if $\mathbf{x}_i$ are columns of $\mathbf{X}$.

Thus:

$$\sum_i \mathbf{x}_i^T\mathbf{R}\mathbf{x}_i = \mathrm{tr}(\mathbf{X}^T\mathbf{R}\mathbf{X})$$

Going back to the original expression, which was:

$$
\frac{1}{\left| 2\pi\mathbf{V}\right|^{N/2}}
\exp \left(
   - \frac{1}{2}
   \sum_i(\mathbf{y}_i - \mathbf{A}\mathbf{x}_i)^T \mathbf{V}^{-1}(\mathbf{y}_i - \mathbf{A}\mathbf{x}_i)
\right)
$$

This can thus be written as:

$$
\frac{1}{\left| 2\pi\mathbf{V}\right|^{N/2}}
\exp \left(
   - \frac{1}{2}
   \mathrm{tr}\left(
      (\mathbf{Y} - \mathbf{A}\mathbf{X})^T \mathbf{V}^{-1}(\mathbf{Y} - \mathbf{A}\mathbf{X})
   \right)
\right)
$$

Since the trace is invariant under cyclic permutations, ie $\mathrm{tr}(ABC)  = \mathrm{tr}(BCA) = \mathrm{tr}(CAB)$, so we can write this as:

$$
=
\frac{1}{\left| 2\pi\mathbf{V}\right|^{N/2}}
\exp \left(
   - \frac{1}{2}
   \mathrm{tr}\left(
      \mathbf{V}^{-1}
      (\mathbf{Y} - \mathbf{A}\mathbf{X})
      (\mathbf{Y} - \mathbf{A}\mathbf{X})^T
   \right)
\right)
$$

$$
=
\frac{1}{\left| 2\pi\mathbf{V}\right|^{N/2}}
\exp \left(
   - \frac{1}{2}
   \mathrm{tr}\left(
      \mathbf{V}^{-1}
      (\mathbf{Y} - \mathbf{A}\mathbf{X})
      (\mathbf{Y}^T - \mathbf{X}^T\mathbf{A}^T)
   \right)
\right)
$$


Multiplying this out, we get:

$$
=
\frac{1}{\left| 2\pi\mathbf{V}\right|^{N/2}}
\exp \left(
   - \frac{1}{2}
   \mathrm{tr}\left(
      \mathbf{V}^{-1}
      \left(
         \mathbf{A}\mathbf{X}\mathbf{X}^T\mathbf{A}^T
         -\mathbf{A}\mathbf{X}\mathbf{Y}^T
         -\mathbf{Y}\mathbf{X}^T\mathbf{A}^T
         + \mathbf{Y}\mathbf{Y}^T
      \right)
   \right)
\right)
$$

$$
=
\frac{1}{\left| 2\pi\mathbf{V}\right|^{N/2}}
\exp \left(
   - \frac{1}{2}
   \mathrm{tr}\left(
      \mathbf{V}^{-1}
      \left(
         \mathbf{A}\mathbf{X}\mathbf{X}^T\mathbf{A}^T
         -2\mathbf{Y}\mathbf{X}^T\mathbf{A}^T
         + \mathbf{Y}\mathbf{Y}^T
      \right)
   \right)
\right)
$$

... which matches the end of section 1 of the Minka paper.

### Matrix-normal distribution

Moving to section 2 of the Minka paper, he writes down the definition of a matrix-normal distribution.  A random $d$ by $m$ matrix $\mathbf{A}$ is matrix-normal distributed with parameters $\mathbf{M}$, $\mathbf{V}$ and $\mathbf{K}$ if the density of $\mathbf{A}$ is:

$$
p(\mathbf{A}) \sim \mathcal{N}(\mathbf{M}, \mathbf{V}, mathbf{K})
$$

$$
=
\frac{\left| \mathbf{K} \right|^{d/2}}
  { \left|2\pi \mathbf{V} \right|^{m/2}}
\exp \left(
  - \frac{1}{2}
  \mathrm{tr}
  \left(
     (\mathbf{A} - \mathbf{M})^T
     \mathbf{V}^{-1}
     (\mathbf{A} - \mathbf{M})
     \mathbf{K}
  \right)
\right)
$$

where:

- $\mathbf{A}$ is $d$ by $m$ (so $m$ is like our $N$, and $d$ is like our $K$; $\mathbf{A}$ has $m$ columns, each of $d$ features)
- $\mathbf{M}$ is $d$ by $m$ too (it I guess represents the mean *of each datapoint*)
- $\mathbf{V}$ is $d$ by $d$ (so, like our $K$ by $K$, seems to plausibly represent our covariance matrix?)
- $\mathbf{K}$ is $m$ by $m$ (like our $N$ by $N$; plausibly represents covariance between data points; for us this should be $\mathbf{I}$ I guess?)

Interestingly, $M$ has as many columns as there are datapoints, so it looks like the mean is per data-point.  This means if the mean of multiple datapoints should be shared, then the values of that mean should be replicated to multiple locations in $\mathbf{M}$, I guess?  It seems like this might be useful to us, if we are going to have a mean that is conditional on a latent class? Or even a mixture of classes/features, which could mean that the mean really is different for each data point?

Minka states that if $\mathbf{K}$ is diagonal, that means the columns of $\mathbf{A}$ are independent normal vectors.  That makes sense, since it is $m$ by $m$, and plausibly represents the covariance between the data points.   I think that $\mathbf{K}$ will be the identity matrix for us, meaning that each data point is drawn $i.i.d.$.

Minka then states that if we vectorize $\mathbf{A}$, stack each column one on top of the other, then the resulting vector will be distributed as:

$$
p(\mathrm{vec}(\mathbf{A}))
= \mathcal{N}
(\mathrm{vec}(\mathbf{M}),
\mathbf{K}^{-1} \otimes \mathbf{V}
)
$$

Intuitively, this kind of makes sense.  $\mathbf{M}$ is the mean per data point.  If we vectorize it, we now have the means stacked one on top of the other.  If $\mathbf{M}$ looks like:

$$
M = [\mathbf{m}_1 \mathbf{m}_2 \dots \mathbf{m}_m]$$

where unfortunately we have $\mathbf{m}_m$ is the $m$th column vector of $\mathbf{M}$ matrix, which conflicting notation I should probably fix, but I think it's kind of manageable for now...

Then, vectorizing this, we'll have:

$$
\mathrm{vec}(\mathbf{M})
=
\begin{bmatrix}
\mathbf{m}_1 \\
\mathbf{m}_2 \\
\dots \\
\mathbf{m}_m \\
\end{bmatrix}
$$

... where $\mathbf{m}_i$ are column vectors.

Then we're going to draw from a Gaussian with this mean.  The covariance for the Gaussian should have the following characteristics:

- each individual point should be independent, meaning each block of $d$ (our $K$) rows of the input/mean should be independent, have no covariance
- each feature within each datapoint should have a covariance matching our desired original covariance, which in this case I think is $\mathbf{V}$.

Then, the Kronecker product $\mathbf{K}^{-1} \otimes \mathbf{V}$ is going to tile $\mathbf{V}$ down the diagonal of the resulting matrix, assuming that $\mathbf{K}$ is the identity matrix $\mathbf{I}$.  The resulting matrix will look something like, where $\mathbf{V}$ is the matrix $\mathbf{V}$, and $\mathbf{0}$ is the empty matrix, filled with $0$s:

$$
\mathbf{K}^{-1} \otimes \mathbf{V}
$$

$$
=
\mathbf{I} \otimes \mathbf{V}
$$

$$
=
\begin{bmatrix}
\mathbf{V} & \mathbf{0} & \dots & \mathbf{0} \\
\mathbf{0} & \mathbf{V} & \dots & \mathbf{0} \\
\dots & \dots & \dots & \dots \\
\mathbf{0} & \mathbf{0} & \dots & \mathbf{V} \\
\end{bmatrix}
$$

Basically, the resulting matrix is block diagonal.  The number of blocks of $\mathbf{V}$ along each dimension will be $m$, corresponding to our $N$.

The covariance matrix corresponds to the requirements listed earlier:

- features within each block of $d$ features are correlated, according to the covariance matrix $\mathbf{V}$
- features between blocks of $d$ features are independent, drawn i.i.d, corresponding to our data point being independently drawn


Deviating somewhat from Minka's paper, given the equivalence of a matrix-normal distribution as a multivariate distribution, this means we can write down the integral of a matrix-normal distribution.  We have, for a multivariate Gaussian:

$$
\int_{-\infty}^{\infty}
\exp \left(
   -\frac{1}{2}
   (\mathbf{x}^T \mathbf{V}^{-1} \mathbf{x})
\right)
\,
d\mathbf{x}
=
\sqrt{\frac{(2\pi)^d}{\mathrm{det}\,\mathbf{V}}}
$$


But in our case we have the matrix-normal distribution:

$$
\int_{-\infty}^{\infty}
\exp \left(
   -\frac{1}{2}
   (\mathbf{X}^T \mathbf{V}^{-1} \mathbf{X})
\right)
\,
d\mathbf{x}
$$

But we should normalize it really.

Oh.... Heh!  .... a matrix-normal distribution is a probability distribution ... it integrates to 1 :-P

So, actually ~~most~~ all of the above is kind of superfluous to knowing the integral of a matrix-normal distribution, since:

- the integral over a matrix-normal distribution is simply $1$, as for any other probability distribution :-), and
- the integral of an un-normalized exponential expression in the form of a matrix-normal, is given by the normalization constant at the start of the matrix-normal expression, right at the start of all of this :-P

### Integrating expressions having the form of a matrix-normal distribution

So:

a matrix-normal distribution has the form:

$$
\mathcal{N}(\mathbf{A}; \mathbf{M}, \mathbf{V}, \mathbf{K})
=
\frac
{\left| \mathbf{K} \right|^{d/2}}
{\left| 2\pi \mathbf{V} \right|^{m/2} }
\exp
\left(
  - \frac{1}{2}
  \mathrm{tr} \left(
    (\mathbf{A} - \mathbf{M})^T
    \mathbf{V}^{-1}
    (\mathbf{A} - \mathbf{M})
    \mathbf{K}
  \right)
\right)
$$

It integrates necessarily to 1, being a probability distribution.  Therefore we can write down directly, based on this, that:

$$
\int_{-\infty}^{\infty}
\exp
\left(
  - \frac{1}{2}
  \mathrm{tr} \left(
    (\mathbf{A} - \mathbf{M})^T
    \mathbf{V}^{-1}
    (\mathbf{A} - \mathbf{M})
    \mathbf{K}
  \right)
\right)
\,d\mathbf{A}
= 
\frac
{\left| 2\pi \mathbf{V} \right|^{m/2} }
{\left| \mathbf{K} \right|^{d/2}}
$$

## Back to finite linear-Gaussian model :-)

We had got as far as writing down the joint probability distribution over $\mathbf{X}$ and $\mathbf{A}$:

$$
\propto
\exp
\left(
- \frac{1}{2}
  \mathrm{tr} \left(
    \frac{1}{\sigma_X^2}\mathbf{X}^T\mathbf{X}
    - \frac{1}{\sigma_X^2} \mathbf{A}^T\mathbf{Z}^T\mathbf{X}
    - \frac{1}{\sigma_X^2} \mathbf{X}^T \mathbf{Z} \mathbf{A}
    + \frac{1}{\sigma_X^2} \mathbf{A}^T\mathbf{Z}^T\mathbf{Z}\mathbf{A}
    + \frac{1}{\sigma_A^2} \mathbf{A}^T \mathbf{A}
  \right)
\right)
$$

$$
= \exp \left(
  - \frac{1}{2}
  \mathrm{tr} \left(
    \frac{1}{\sigma_X^2}\mathbf{X}^T\mathbf{X}
    - \frac{1}{\sigma_X^2} \mathbf{A}^T\mathbf{Z}^T\mathbf{X}
    - \frac{1}{\sigma_X^2} \mathbf{X}^T \mathbf{Z} \mathbf{A}
    + \mathbf{A}^T(\frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z} + \frac{1}{\sigma_A^2}\mathbf{I})\mathbf{A}
  \right)
\right)
$$

$$
= \exp \left(
  - \frac{1}{2}
  \mathrm{tr} \left(
    \frac{1}{\sigma_X^2}\mathbf{X}^T\mathbf{X}
    - (\frac{2}{\sigma_X^2}\mathbf{X}^T \mathbf{Z})\mathbf{A}
    + \mathbf{A}^T(\frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z} + \frac{1}{\sigma_A^2}\mathbf{I})\mathbf{A}
  \right)
\right)
\tag{4.1}
$$

The tutorial then completes the square for $\mathbf{A}$ which I supposed/suppose is because we want to integrate over it shortly?



### Complete the square

To complete the square, let's examine what happens if we multiply an expression like:

$$(\mathbf{A}\mathbf{X} - \mathbf{B}\mathbf{Y})^T\mathbf{S}(\mathbf{A}\mathbf{X} - \mathbf{B}\mathbf{Y})$$

$$= (\mathbf{X}^T\mathbf{A}^T\mathbf{S} - \mathbf{Y}^T\mathbf{B}^T\mathbf{S})(\mathbf{A}\mathbf{X} - \mathbf{B}\mathbf{Y})$$

$$=\mathbf{X}^T\mathbf{A}^T\mathbf{S}\mathbf{A}\mathbf{X}
-\mathbf{Y}^T\mathbf{B}^T\mathbf{S}\mathbf{A}\mathbf{X}
-\mathbf{X}^T\mathbf{A}^T\mathbf{S}\mathbf{B}\mathbf{Y}
+\mathbf{Y}^T\mathbf{B}^T\mathbf{S}\mathbf{B}\mathbf{Y}$$

$$
=\mathbf{X}^T\mathbf{A}^T\mathbf{S}\mathbf{A}\mathbf{X}
-2\mathbf{X}^T\mathbf{A}^T\mathbf{S}\mathbf{B}\mathbf{Y}
+ \mathbf{Y}^T\mathbf{B}^T\mathbf{S}\mathbf{B}\mathbf{Y}
$$

In our case, the constant term on the left is not a constraint we need to fit, since we will complete the square, but we should create an expression whose resulting terms in $\mathbf{Y}$ and $\mathbf{Y}^T\dots\mathbf{Y}$, which in our case are expressions in $\mathbf{A}$, matches the corresponding terms inside equation (4.1).

Since $(\frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z} + \frac{1}{\sigma_A^2}\mathbf{I})$ is not obviously decomposable into the product of a matrix and its transpose, so this should probably be in the $\mathbf{S}$ value.

We can then have $\mathbf{B}$ as the identity matrix, and $\mathbf{X}^T\mathbf{A}^T$ together can be:

$$
\frac{1}{\sigma_X^2} \mathbf{X}^T\mathbf{Z} \left( \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z} + \frac{1}{\sigma_A^2}\mathbf{I} \right)^{-1}
$$

So, $\mathbf{A}\mathbf{X}$ is:

$$
\frac{1}{\sigma_X^2}
\left(
  \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z} + \frac{1}{\sigma_A^2}\mathbf{I}
\right)^{-1^T}
\mathbf{Z}^T\mathbf{X}
$$

### Interlude: Gramian matrix is symmetric

Let's suppose we have:

$$
\mathbf{B} = \mathbf{A}^T\mathbf{A}
$$

(where $A$ is $m$ x $n$)

Then:

$$
b_{i,j} = \sum_{k=1}^m a_{k,i} a_{k,j}
$$

Conversely:

$$
(\mathbf{B}^T)_{i,j} = b_{j,i} = \sum_{k=1}^m a_{k,j} a_{k,i}
$$

$$
= \sum_{k=1}^m a_{k,i} a_{k,j}
$$

$$
= b_{i,j}
$$

So:

$$\mathbf{A}^T\mathbf{A} = (\mathbf{A}^T\mathbf{A})^T$$

### Expression for joint probability of $\mathbf{X}$ and $\mathbf{A}$ continued

Therefore, for our $\mathbf{A}\mathbf{X}$:

$$
\mathbf{A}\mathbf{X}
=
\frac{1}{\sigma_X^2}
\left(
  \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z} + \frac{1}{\sigma_A^2}\mathbf{I}
\right)^{-1^T}
\mathbf{Z}^T\mathbf{X}
$$

$$
= 
\frac{1}{\sigma_X^2}
\left(
  \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z} + \frac{1}{\sigma_A^2}\mathbf{I}
\right)^{-1}
\mathbf{Z}^T\mathbf{X}
$$

Let's define:

$$\mathbf{S} =
\left(
    \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z}
    +
    \frac{1}{\sigma_A^2}\mathbf{I}
\right)
$$


And so the expression inside (4.1) becomes:

$$
\frac{1}
  {\sigma_X^2}
  \mathbf{X}^T\mathbf{X}
- \frac{1}{\sigma_X^2}\mathbf{X}^T\mathbf{Z}\mathbf{S}^{-1}
  \mathbf{S}
  \frac{1}{\sigma_X^2}\mathbf{X}
  \mathbf{S}^{-1} \mathbf{Z}^T\mathbf{X}
+ \left( \frac{1}{\sigma_X^2}\mathbf{S}^{-1}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)^T
\mathbf{S}
\left( \frac{1}{\sigma_X^2}\mathbf{S}^{-1}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)
$$


$$
=
\frac{1}
  {\sigma_X^2}
  \mathbf{X}^T\mathbf{X}
- \frac{1}{\sigma_X^4}\mathbf{X}^T\mathbf{Z}
  \mathbf{X}
  \mathbf{S}^{-1} \mathbf{Z}^T\mathbf{X}
+ \left( \frac{1}{\sigma_X^2}\mathbf{S}^{-1}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)^T
\mathbf{S}
\left( \frac{1}{\sigma_X^2}\mathbf{S}^{-1}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)
$$

The tutorial then defines:

$$
\mathbf{M}
= \frac{ \mathbf{S}^{-1}}{\sigma_X^2}
$$

which gives:

$$
\frac{1}
  {\sigma_X^2}
  \mathbf{X}^T\mathbf{X}
- \frac{1}{\sigma_X^2}\mathbf{X}^T\mathbf{Z}
  \mathbf{X}
  \mathbf{M} \mathbf{Z}^T\mathbf{X}
+ \left( \mathbf{M}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)^T
(\sigma_X^2\mathbf{M})^{-1}
\left( \mathbf{M}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)
$$



$$
=
\frac{1}
  {\sigma_X^2}
  \mathbf{X}^T
  \left(
     \mathbf{I}
    - \mathbf{Z}\mathbf{M} \mathbf{Z}^T
  \right)
  \mathbf{X}
+ \left( \mathbf{M}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)^T
(\sigma_X^2\mathbf{M})^{-1}
\left( \mathbf{M}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)
$$

... which matches the tutorial

### Marginalize over $\mathbf{A}$

We want to marginalize over $\mathbf{A}$, to obtain:

$$
p(\mathbf{X} \mid \mathbf{Z}, \sigma_X, \sigma_A)
$$

This is calculated as:

$$
\int
p(\mathbf{X} \mid \mathbf{Z}, \mathbf{A}, \sigma_X)
p(\mathbf{A} \mid \sigma_A) \, d\mathbf{A}
$$


$$
=
\frac{1}
  {(2\pi \sigma_X^2)^{ND/2}}
  \frac{1}
  {(2\pi \sigma_A^2)^{KD/2}}
\int
\exp \left(
- \frac{1}{2}
\mathrm{tr} \left(
      \frac{1}
      {\sigma_X^2}
      \mathbf{X}^T
      \left(
         \mathbf{I}
        - \mathbf{Z}\mathbf{M} \mathbf{Z}^T
      \right)
      \mathbf{X}
    + \left( \mathbf{M}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)^T
    (\sigma_X^2\mathbf{M})^{-1}
    \left( \mathbf{M}\mathbf{Z}^T\mathbf{X} - \mathbf{A} \right)
   \right)
\right)
\,
d\mathbf{A}
$$

...and it's becoming obvious now why $\mathbf{M}$ is defined as it is, and why we completed the square for $\mathbf{A}$.  To compute the integral over the matrix Gaussian form, we need ideally $\mathbf{A}$ to not have any coefficients etc, and we need ideally the variance term, in this case $\sigma_X^2\mathbf{M}$, to be factorized out, in the way that it now is.  We can factorize the exponential into two parts, moving the term that is independent of $\mathbf{A}$ to outside of the integral:

$$
=
\frac{1}
  {(2\pi \sigma_X^2)^{ND/2}}
  \frac{1}
  {(2\pi \sigma_A^2)^{KD/2}}
\exp \left(
    - \frac{1}{2}
    \mathrm{tr} \left(
      \frac{1}{\sigma_X^2}
      \mathbf{X}^T
      (\mathbf{I} - \mathbf{Z}\mathbf{M} \mathbf{Z}^T )
      \mathbf{X}
   \right)
\right)
\int
\exp \left(
   - \frac{1}{2}
   \mathrm{tr} \left(
        ( \mathbf{M}\mathbf{Z}^T\mathbf{X} - \mathbf{A} )^T
        (\sigma_X^2\mathbf{M})^{-1}
        ( \mathbf{M}\mathbf{Z}^T\mathbf{X} - \mathbf{A} )
   \right)
\right)
\,
d\mathbf{A}
$$

For the matrix-normal form, the integral is just the inverse of normalizing constant of a matrix-normal distribution ie, for the notation used in the Minka paper:

$$
\frac
  {\left| 2\pi \mathbf{V} \right|^{m/2}}
  {\left| \mathbf{K} \right|^{d/2}}
$$

In our case, for the marginalize integral above, $\mathbf{K}$ is $\mathbf{I}$, and $\mathbf{V}$ is $\sigma_X^2\mathbf{M}$ $m$ is the number of columns of $\mathbf{A}$, which is $D$.  So the integral is:

$$
\frac
  {\left| 2\pi \mathbf{\sigma_X^2\mathbf{M}} \right|^{D/2}}
  {1}
$$


So the marginalized form becomes:

$$
=
\frac{\left| 2\pi \mathbf{\sigma_X^2\mathbf{M}} \right|^{D/2}}
  { (2\pi \sigma_X^2)^{ND/2}  (2\pi \sigma_A^2)^{KD/2}}
\exp \left(
    - \frac{1}{2}
    \mathrm{tr} \left(
      \frac{1}{\sigma_X^2}
      \mathbf{X}^T
      (\mathbf{I} - \mathbf{Z}\mathbf{M} \mathbf{Z}^T )
      \mathbf{X}
   \right)
\right)
$$

Looking at the normalization term:

$$
\frac{\left| 2\pi \mathbf{\sigma_X^2\mathbf{M}} \right|^{D/2}}
  { (2\pi \sigma_X^2)^{ND/2}  (2\pi \sigma_A^2)^{KD/2}}
$$

### Interlude: scalar multiple of a matrix determinant

So, at this point, I realized that Minka's normalization constnat has power of $d/2$, using his notation, whereas the Gaussian for $\mathbf{X}$ in the tutorial has $ND/2$.   So, let's look at what's happening:

For the general matrix-normal distribution, from Minka's report, the normalization constant is:

$$
\frac
   {\left|\mathbf{K}\right|^{N/2}}
   {\left| 2\pi \mathbf{V}\right|^{D/2}}
$$
$$
= 
\frac
   {\left|\mathbf{K}\right|^{N/2}}
   {(2\pi)^{ND/2} \left| \mathbf{V}\right|^{D/2}}
$$

Here:

$$\mathbf{K} = \mathbf{I}$$

and:

$$\mathbf{V} = \sigma_X^2 \mathbf{I}$$

For a determinant, we have:

$$|c\mathbf{A}| = c^n|\mathbf{A}|$$

...where $\mathbf{A}$ is a square matrix of dimension $n$ x $n$

I found this from Wikipedia, https://en.wikipedia.org/wiki/Determinant , but we can easily intuit on this:

- a determinant can be written as a linear of each value in the first row of the matrix, multiplied by the determinant of a minor
- multiplying the matrix by a scalar will increase the magnitude of the values of the first row by this scalar, thus multiply the determinant by this scalar
- however, it will also have a similar effect on the determinant of each minor
- by induction, the magnitude of the determinant will be increased by $c^n$, as above

So:

$$
\left|\mathbf{V}\right| = (\sigma_X^2)^N
$$


And:

$$
\frac
  {\left| \mathbf{K} \right|^{N/2}}
  {\left| 2\pi \mathbf{V} \right|^{D/2}}
=
\frac
  {1}
  {(2\pi\sigma_X^2)^{ND/2}}
$$

... which matches the tutorial.

### Constant of normalization, for marginalization over $\mathbf{A}$

So, going back to the marginalized probability, the constant of normalization so far is:

$$
\frac{\left| 2\pi \mathbf{\sigma_X^2\mathbf{M}} \right|^{D/2}}
  { (2\pi \sigma_X^2)^{ND/2}  (2\pi \sigma_A^2)^{KD/2}}
$$

$\mathbf{M}$ is:

$$\mathbf{M} = \frac{\mathbf{S}^{-1}}{\sigma_X^2}$$

$$= \frac{{\left(
    \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z}
    +
    \frac{1}{\sigma_A^2}\mathbf{I}
\right)
}^{-1}}{\sigma_X^2}$$

For a determinant, we have:

$$|\mathbf{A}^{-1}| = \frac{1}{|\mathbf{A}|}$$

(Where $\mathbf{A}$ is an invertable matrix)

So:

$$
|\mathbf{M}| = \left| \frac{{\left(
    \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z}
    +
    \frac{1}{\sigma_A^2}\mathbf{I}
\right)
}^{-1}}{\sigma_X^2}
\right|
$$

$$
=
\left| \frac
  {1}
  {\sigma_X^2 \left(
    \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z}
    +
    \frac{1}{\sigma_A^2}\mathbf{I}
  \right)
}
\right|
$$

$$
=
 \frac
  {1}
  {\left|
    \mathbf{Z}^T\mathbf{Z}
    +
    \frac{\sigma_X^2 }{\sigma_A^2}\mathbf{I}
  \right|
}
$$

We will substitute in the value of $|\mathbf{M}|$ into the expression for the normalization constant.  Wd note that $\mathbf{Z}$ is $N$ x $K$, and therefore $\mathbf{Z}^T\mathbf{Z}$ is $K$ x $K$.  So, we get:

$$
\frac{ (2\pi \sigma_X^2)^{KD/2} }
  { (2\pi)^{\frac{ND + KD}{2}} ( \sigma_X^2)^{ND/2}  ( \sigma_A^2)^{KD/2}
    \left|
    \mathbf{Z}^T\mathbf{Z}
    +
    \frac{\sigma_X^2 }{\sigma_A^2}\mathbf{I}
    \right|^{D/2}
  }
$$



$$
=
\frac{ 1 }
  { (2\pi)^{\frac{ND}{2}} ( \sigma_X^2)^{ND/2 - KD/2}  ( \sigma_A^2)^{KD/2}
    \left|
    \mathbf{Z}^T\mathbf{Z}
    +
    \frac{\sigma_X^2 }{\sigma_A^2}\mathbf{I}
    \right|^{D/2}
  }
$$

$$
=
\frac{ 1 }
  { (2\pi)^{\frac{ND}{2}} \sigma_X^{(N - K)D}  \sigma_A^{KD}
    \left|
    \mathbf{Z}^T\mathbf{Z}
    +
    \frac{\sigma_X^2 }{\sigma_A^2}\mathbf{I}
    \right|^{D/2}
  }
$$

... which matches the tutorial

### Exponential trace expression for marginalization over $\mathbf{A}$

As far as the expression inside the exponential trace, we have:

$$
      \frac{1}{\sigma_X^2}
      \mathbf{X}^T
      (\mathbf{I} - \mathbf{Z}\mathbf{M} \mathbf{Z}^T )
      \mathbf{X}
$$


And:

$$\mathbf{M} = \frac{{\left(
    \frac{1}{\sigma_X^2}\mathbf{Z}^T\mathbf{Z}
    +
    \frac{1}{\sigma_A^2}\mathbf{I}
\right)
}^{-1}}{\sigma_X^2}
$$

For a matrix inverse we have:

$$(c\mathbf{A})^{-1} = \frac{1}{c}\mathbf{A}^{-1}$$

... for scalar $c$, and invertible matrix $\mathbf{A}$

So:

$$
\mathbf{M} =
\left(
    \mathbf{Z}^T\mathbf{Z}
    +
    \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I}
\right)^{-1}
$$

And, substituting into the exponential trace expression, we get:

$$
      \frac{1}{\sigma_X^2}
      \mathbf{X}^T
      (\mathbf{I} - \mathbf{Z}\mathbf{M} \mathbf{Z}^T )
      \mathbf{X}
$$

$$
= 
      \frac{1}{\sigma_X^2}
      \mathbf{X}^T
      (\mathbf{I} - \mathbf{Z}\left(
    \mathbf{Z}^T\mathbf{Z}
    +
    \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I}
\right)^{-1} \mathbf{Z}^T )
      \mathbf{X}
$$

... as required.

### Conditional expression for one latent element

For Gibbs sampling, we need the conditional probability of one latent element $z_{i,k}$, conditional on the other latent elements, and also on the observed data values.

ie, we need:

$$
P(z_{i,k} \mid \mathbf{X}, \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)
$$

Note that since we are marginalizing over $\mathbf{A}$, there is no $\mathbf{A}$ in this expression.

By Bayes Rules, the conditional probability for $z_{i,k}$ is:

$$
P(z_{i,k} | \mathbf{X}, \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)
=
\frac
  {P(\mathbf{Z}_{-(i,k)} \mid z_{i,k}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k} \mid \mathbf{X}, \sigma_X, \sigma_A)}
  {P(\mathbf{Z}_{-(i,k)} \mid \mathbf{X}, \sigma_X, \sigma_A)}
$$

$$
=
\frac
  {P(\mathbf{Z}_{-(i,k)} \mid z_{i,k}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k})}
  {P(\mathbf{Z}_{-(i,k)}}
$$

or, we can try slightly differently:

$$
P(z_{i,k} | \mathbf{X}, \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)
=
\frac
  {P(\mathbf{X} \mid z_{i,k}, \mathbf{Z}_{-(i,k)}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k} \mid \mathbf{Z}_{-(i,k)},\sigma_X, \sigma_A)}
  {P(\mathbf{X} \mid \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)}
$$

$$
=
\frac
  {P(\mathbf{X} \mid \mathbf{Z}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k} \mid \mathbf{Z}_{-(i,k)},\sigma_X, \sigma_A)}
  {P(\mathbf{X} \mid \mathbf{Z}_{-(i,k)}, \sigma_X, \sigma_A)}
$$

$$
\propto P(\mathbf{X} \mid \mathbf{Z}, \mathbf{X}, \sigma_X, \sigma_A)\,P(z_{i,k} \mid \mathbf{Z}_{-(i,k)},\sigma_X, \sigma_A)
$$

(since the denominator is constant wrt $z_{i,k}$, therefore doesnt change the shape of the probability distribution of $z_{i,k}$, simply acts as a constant of normalization)

$P(\mathbf{X} \mid \mathbf{X}, \sigma_X, \sigma_A)$, we've just calculated, by marginalizing the joint probability of $\mathbf{X}$ and $\mathbf{A}$ over $\mathbf{A}$.

$P(z_{i,k} \mid \mathbf{Z}_{-(i,k)})$, we calculated in the previous section, for Gibbs sampling of a finite feature model:

$$
P(z_{i,k} \mid \mathbf{z}_{-i,k})
= \frac{m_{-i,k} + \frac{\alpha}{K}}
  {N + \frac{\alpha}{K}}
$$


### Interlude: rank-1 updates

The formula for the exponential trace expression of the marginalized form over $\mathbf{A}$ needs the calculation of an inverse:

$$
\left(
  \mathbf{Z}^T\mathbf{Z}
  + \frac{\sigma_X^2}{\sigma_A^2}
  \mathbf{I}
\right)^{-1}
$$

Matrix inversions are generally expensive, computationally. For example, if a matrix is symmetric and positive-definite, one can use Cholesky Decomposition to calculate the inverse, which uses about $\frac{1}{3}n^3$ flops, where $n$ is the size of the matrix.  This is less than LU decomposition, which uses about $\frac{2}{3}n^3$ flops, but is still cubic in $n$.

Therefore if we can avoid re-calculating an inverse repeatedly during an algorithmic implementation, then the implementation will run faster.

One way to avoid calculating the inverse repeatedly, is to use rank-1 updates, where this is possible.

Given the Cholesky decomposition of matrix $\mathbf{A}$ into $\mathbf{L}\mathbf{L}^T$, where $\mathbf{L}$ is a lower triangular matrix, then the rank-1 update calculates $\mathbf{L}'$ where:

$$
\mathbf{A} + \mathbf{x}\mathbf{x}^T
= \mathbf{L}' \mathbf{L}'^T
$$

...given vector $\mathbf{x}$, and the original decomposition of $\mathbf{A} = \mathbf{L}\mathbf{L}^T$

Per "Methods for Modifying Matrix Factorizations", by Gill et al, 1974, rank-1 update to Cholesky factorization can be carried out in $O(n^2)$ flops. (method 'C1').

From a practical, engineering, point of view, it looks like numpy doesnt provide an implementation for rank-1 updates.  LINPACK does, and matlab does, but numpy doesnt.

[Wikipedia](https://en.wikipedia.org/wiki/Cholesky_decomposition) provides a matlab implementation of a rank-1 update.  Let's try implementing it in Python, and then try a `numpy` built-in method (which presumably uses BLAS)

In [15]:
import numpy as np
import math
import time

N = 3000
B = np.random.randn(N, N)
A = B.T.dot(B)
print(A.shape)
print(A[:2,:2])
start = time.time()
L = np.linalg.cholesky(A)
print('time for full rank Cholesky %s seconds' % (time.time() - start))
print(L[:3,:3])

# check
LLT = L.dot(L.T)
assert np.abs(A - LLT).max() < 1e-6

x = np.random.randn(N)
print(x.shape)
print(x[:3])

def update_cholesky(L, x):
    N = x.shape[0]
    for k in range(N):
        r = math.sqrt(L[k, k] * L[k, k] + x[k] * x[k])
        c = r / L[k, k]
        s = x[k] / L[k, k]
        L[k, k] = r
        L[k + 1:, k] = (L[k + 1:, k] + s * x[k + 1:]) / c
        x[k + 1:] = c * x[k + 1:] - s * L[k + 1:, k]

update_cholesky(L, x)
LLT2 = L.dot(L.T)
diff = np.abs(A + x.dot(x.T) - LLT2).max()
print(diff)
assert diff < 1e-8

(3000, 3000)
[[ 2970.41996429    21.31644394]
 [   21.31644394  2942.49052004]]
time for full rank Cholesky 0.3525505065917969 seconds
[[ 54.50155928   0.           0.        ]
 [  0.39111622  54.243318     0.        ]
 [  0.93970653   0.2525172   54.49271706]]
(3000,)
[-0.14441793  0.18327688 -1.23138222]
3007.10831474


AssertionError: 

... fails, not sure why.  But anyway, looking at the Griffiths and Ghahramani tutorial again, it looks like it's not actually using a generic rank-1 Cholesky update: they're designing/deriving their own rank-1 update formulae.  So let's go through that.

### rank-1 updates to marginal probability of $\mathbf{X}$

Reminder: $\mathbf{M}$ is:

$$\mathbf{M}
= \left(
  \mathbf{Z}^T\mathbf{Z} + \frac{\alpha_X^2}{\alpha_A^2}
  \mathbf{I}
\right)^{-1}
$$

Per the tutorial, we should define:

$$
\mathbf{M}_{-i}
= \left(
\sum_{j \ne i}
z_j^T z_j
+ \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I}
\right)
^{-1}
$$

And then, I cant quite guess/see what the tutorial is/is going to do, so I'm just going to work through what the tutorial does:

$$
\mathbf{M}_{-i}
= (\mathbf{M}^{-1} - \mathbf{z}_i^T\mathbf{z}_i)^{-1}
$$

$$
= ( \dots
$$

### Sheman-Morrison Formula for rank-1 updates

I stared at this for a while, and then decided the tutorial must probably be using a generic, well-known, rank-1 update formula.  I googled for 'matrix inverse rank-1 update', and found:

https://en.wikipedia.org/wiki/Sherman%E2%80%93Morrison_formula

This gives the following formula:

$$
\left(
  \mathbf{A} + \mathbf{u}\mathbf{v}^T 
\right)^{-1}
=
\mathbf{A}^{-1}
-
\frac{\mathbf{A}^{-1}\mathbf{u}\mathbf{v}^T\mathbf{A}^{-1}}
  {1 + \mathbf{v}^T\mathbf{A}^{-1}\mathbf{u}}
$$

This looks pretty similar to equation (23) in the tutorial, so let's work with the idea that the tutorial is using the Sherman-Morrison formula for now.  Let's first modify the Sherman-Morrison formula in two ways:

- write it for a downdate, ie for $(\mathbf{A} - \dots)^{-1}$ rather than $(\mathbf{A} + \dots)^{-1}$, and
- modify to write for the case that there is only one vector $\mathbf{v}$, rather than the more general form of two unequal vectors $\mathbf{u}$ and $\mathbf{v}$

We can do this by substituting $\mathbf{u} = - \mathbf{v}$:

$$
(\mathbf{A} - \mathbf{v}\mathbf{v}^T)^{-1}
=
\mathbf{A}^{-1}
-
\frac
  {\mathbf{A}^{-1} \cdot (-\mathbf{v}) \cdot \mathbf{v}^T \cdot \mathbf{A}^{-1}}
  {1 + \mathbf{v}^T \cdot \mathbf{A}^{-1} \cdot (-\mathbf{v})}
$$

$$
=
\mathbf{A}^{-1}
+
\frac
  {\mathbf{A}^{-1} \cdot (\mathbf{v}) \cdot \mathbf{v}^T \cdot \mathbf{A}^{-1}}
  {1 - \mathbf{v}^T \cdot \mathbf{A}^{-1} \cdot (\mathbf{v})}
$$

Or, by comparison with what is in the tutorial, giving some idea of where this might be going, reversing the order of the terms in the denominator, and the sign of the right-hand side:

$$
=
\mathbf{A}^{-1}
-
\frac
  {\mathbf{A}^{-1} \cdot (\mathbf{v}) \cdot \mathbf{v}^T \cdot \mathbf{A}^{-1}}
  {\mathbf{v}^T \cdot \mathbf{A}^{-1} \cdot (\mathbf{v}) - 1}
$$

$$
=
\mathbf{A}^{-1}
-
\frac
  {\mathbf{A}^{-1} \mathbf{v} \mathbf{v}^T \mathbf{A}^{-1}}
  {\mathbf{v}^T \mathbf{A}^{-1} \mathbf{v} - 1}
$$

This kind of matches what's in the tutorial, except that we have inverses here, like $\mathbf{A}^{-1}$, whereas the tutorial has $\mathbf{M}$, but since $\mathbf{M}$ is in fact defined as an inverse, ie $(\mathbf{Z}^T\mathbf{Z} - \frac{\alpha_X^2}{\alpha_A^2}\mathbf{I})^{-1}$, then this might work out.



Let's substitute in the terms/notation for our actual concrete expression, into the modified Sherman-Morrison formula.  This means we will subsitute $\mathbf{A}$ by $\mathbf{M}^{-1}$, and $\mathbf{v}$ with $\mathbf{z}_i^T$.  Note that $\mathbf{v}$ is a column vector, whereas $\mathbf{z}_i$ is a row vector.  $\mathbf{z}_i^T$ is a column vector, which matches therefore the form of $\mathbf{v}$.  So we get:

$$
(\mathbf{M}^{-1} - \mathbf{z}_i^T\mathbf{z})^{-1}
=
\mathbf{M}
-
\frac{\mathbf{M}\mathbf{z}_i^T\mathbf{z}_i \mathbf{M}}
  {\mathbf{z}_i\mathbf{M}\mathbf{z}_i^T - 1}
$$

... which matches exactly the tutorial.

### Try Sherman-Morrison in numpy

Let's retry rank-1 updates in numpy, using the Sherman-Morrison formula.  Note that the Sherman-Morrison formula doesnt need decomposition, eg QR or Cholesky, just needs the inverse.

In [25]:
import numpy as np
import math
import time

N = 3000
B = np.random.randn(N, N)
A = B.T.dot(B)
print(A.shape)
print(A[:2,:2])
start = time.time()
Ainv = np.linalg.inv(A)
print('time for full rank inverse %s seconds' % (time.time() - start))
print(Ainv[:3,:3])

# check
A_Ainv = A.dot(Ainv)
diff = np.abs(A_Ainv - np.identity(N)).max()
print(diff)
assert diff < 1e-8

x = np.random.randn(N, 1)
print(x.shape)
print(x[:3])

def sherman_morrison(Ainv, x):
    numerator = Ainv.dot(x).dot(x.T).dot(Ainv)
    denominator = x.T.dot(Ainv).dot(x) - 1
    return Ainv - numerator / denominator

start = time.time()
Ainv2 = sherman_morrison(Ainv, x)
print('time for sherman morrison %s seconds' % (time.time() - start))
A2 = A + x.dot(x.T)
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after rank 1 update: %s' % diff)
# assert diff < 1e-8

# try full rank inverse
start = time.time()
Ainv2 = np.linalg.inv(A2)
print('time for full rank inverse %s seconds' % (time.time() - start))
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after full rank inverse %s' % diff)


(3000, 3000)
[[ 2982.31866914    96.38717338]
 [   96.38717338  3039.32098946]]
time for full rank inverse 2.2654874324798584 seconds
[[ 1.36423992 -0.47994222  0.13859518]
 [-0.47994222  0.30385504 -0.01174672]
 [ 0.13859518 -0.01174672  0.33828845]]
6.32098817732e-11
(3000, 1)
[[-1.22683011]
 [ 1.1851708 ]
 [ 0.35445188]]
time for sherman morrison 1.4911704063415527 seconds
diff after rank 1 update: 0.312939937953
time for full rank inverse 2.178471326828003 seconds
diff after full rank inverse 2.45563569479e-11


Hmmmm, Sherman-Morrison took only 40% less time than full-rank, but vastly more inaccurate.

But ... what if we re-arrange the order of the matrix multiplication slightly?

In [26]:
def sherman_morrison_v2(Ainv, x):
    n1 = Ainv.dot(x)
    n2 = x.T.dot(Ainv)
    numerator = n1.dot(n2)
    denominator = x.T.dot(Ainv).dot(x) - 1
    return Ainv - numerator / denominator

start = time.time()
Ainv2 = sherman_morrison_v2(Ainv, x)
print('time for sherman morrison %s seconds' % (time.time() - start))
A2 = A + x.dot(x.T)
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after rank 1 update: %s' % diff)
# assert diff < 1e-8

# try full rank inverse
start = time.time()
Ainv2 = np.linalg.inv(A2)
print('time for full rank inverse %s seconds' % (time.time() - start))
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after full rank inverse %s' % diff)


time for sherman morrison 0.23725247383117676 seconds
diff after rank 1 update: 0.312939937938
time for full rank inverse 2.1450388431549072 seconds
diff after full rank inverse 2.45563569479e-11


Better: still looking pretty inaccurate, compared to full-rank, but at least it's ~10 times faster, for matrices with side $3000$.

After reading the section entitled 'Application' on the wikipedia page, which describes the assymptotic complexity, it occurred to me that `n1` and `n2` are basically the same matrix, so we can do:


In [28]:
def sherman_morrison_v3(Ainv, x):
    n1 = Ainv.dot(x)
    numerator = n1.dot(n1.T)
    denominator = x.T.dot(Ainv).dot(x) - 1
    return Ainv - numerator / denominator

start = time.time()
Ainv2 = sherman_morrison_v3(Ainv, x)
print('time for sherman morrison %s seconds' % (time.time() - start))
A2 = A + x.dot(x.T)
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after rank 1 update: %s' % diff)
# assert diff < 1e-8

# try full rank inverse
start = time.time()
Ainv2 = np.linalg.inv(A2)
print('time for full rank inverse %s seconds' % (time.time() - start))
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after full rank inverse %s' % diff)


time for sherman morrison 0.2752869129180908 seconds
diff after rank 1 update: 0.312939914919
time for full rank inverse 2.1179018020629883 seconds
diff after full rank inverse 2.45563569479e-11


...but the time is the same.  So presumably the time is dominated by the `n1.dot(n1.T)` calculation.  Check this point:

In [43]:
def sherman_morrison_with_timing(Ainv, x):
    last = time.time()
    n1 = Ainv.dot(x)
    print('time for n1 %s' % (time.time() - last))
    last = time.time()
    numerator = n1.dot(n1.T)
    print('time for numerator %s' % (time.time() - last))
    last = time.time()
    denominator = x.T.dot(Ainv).dot(x) - 1
    print('time for denominator %s' % (time.time() - last))
    last = time.time()
    res = Ainv - numerator / denominator
    print('time for final per-element %s' % (time.time() - last))
    return res

start = time.time()
Ainv2 = sherman_morrison_with_timing(Ainv, x)
print('time for sherman morrison %s seconds' % (time.time() - start))
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after rank 1 update: %s' % diff)


time for n1 0.0075109004974365234
time for numerator 0.052231788635253906
time for denominator 0.010253190994262695
time for final per-element 0.06621360778808594
time for sherman morrison 0.138261079788208 seconds
diff after rank 1 update: 0.312939914919


Dominated by the final per-element operations, interestingly.  And surprisingly.

Is this because the per-element scalar division is being implemented using division, rather than as multiplication by 1 over the divisor?  Check this point:

In [44]:
def sherman_morrison_with_timing_v2(Ainv, x):
    last = time.time()
    n1 = Ainv.dot(x)
    print('time for n1 %s' % (time.time() - last))
    last = time.time()
    numerator = n1.dot(n1.T)
    print('time for numerator %s' % (time.time() - last))
    last = time.time()
    denominator = x.T.dot(Ainv).dot(x) - 1
    print('time for denominator %s' % (time.time() - last))
    last = time.time()
    res = Ainv - numerator * (1 / denominator)
    print('time for final per-element %s' % (time.time() - last))
    return res

start = time.time()
Ainv2 = sherman_morrison_with_timing_v2(Ainv, x)
print('time for sherman morrison %s seconds' % (time.time() - start))
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after rank 1 update: %s' % diff)


time for n1 0.008286476135253906
time for numerator 0.07381415367126465
time for denominator 0.011293649673461914
time for final per-element 0.12431550025939941
time for sherman morrison 0.2201707363128662 seconds
diff after rank 1 update: 0.312939914921


Worse. bizarrely.  It's almost as though the operations are being executed asynchonously, and the final operation is a sync point.  It is surprising for per-element operations to take relatively significant time.

Is it something like, the `denominator` value is a matrix, and somehow this makes it take longer?  Let's convert it to a scalar:

In [45]:
def sherman_morrison_with_timing_v2(Ainv, x):
    last = time.time()
    n1 = Ainv.dot(x)
    print('time for n1 %s' % (time.time() - last))
    last = time.time()
    numerator = n1.dot(n1.T)
    print('time for numerator %s' % (time.time() - last))
    last = time.time()
    denominator = x.T.dot(Ainv).dot(x) - 1
    print('time for denominator %s' % (time.time() - last))
    last = time.time()
    res = Ainv - numerator * (1 / denominator.item())
    print('time for final per-element %s' % (time.time() - last))
    return res

start = time.time()
Ainv2 = sherman_morrison_with_timing_v2(Ainv, x)
print('time for sherman morrison %s seconds' % (time.time() - start))
A2_Ainv2 = A2.dot(Ainv2)
diff = np.abs(A2_Ainv2 - np.identity(N)).max()
print('diff after rank 1 update: %s' % diff)


time for n1 0.009202003479003906
time for numerator 0.05918169021606445
time for denominator 0.008460283279418945
time for final per-element 0.06301307678222656
time for sherman morrison 0.14108610153198242 seconds
diff after rank 1 update: 0.312939914921


Weirdly, making the denominator explicitly a scalar accelerates the per-element operations by 50%, and now the matrix multiplication of the numerator dominates.

The Sherman-Morrison update is now about ~15 times faster than the full-rank inverse, though still just as imprecise.

Going back to the tutorial, we still need the expression for $\mathbf{M}$, in terms of $\mathbf{M}_{-i}$, but it is clear that it is just Sherman-Morrison again, the exact same formula, just with slightly different inputs.

### Taking the infinite limit

Just following through the tutorial directly:

We take $\mathbf{Z}$ to be in left-ordered form, and write it as $[ \mathbf{Z}_+ \mathbf{Z}_0]$.

Therefore, $|\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I}|$ becomes:

$$
\left|
  \begin{bmatrix}
    \mathbf{Z}^T_+ \mathbf{Z}_+ & \mathbf{0} \\
    \mathbf{0} & \mathbf{0} \\
  \end{bmatrix}
  +
  \frac{\sigma_X^2}{\sigma_A^2}
  \mathbf{I}_K
\right|
$$

$$
= \left(
  \frac{\sigma_X^2}{\sigma_A^2}
\right)^{K_0}
\left|
  \mathbf{Z}_+^T\mathbf{Z}_+
  +
  \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I}_{K^+}
\right|
$$

And: $\mathbf{Z}(\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}\mathbf{I})^{-1}\mathbf{Z}^T$ becomes:

$$
\mathbf{Z}_+
\left(
\mathbf{Z}^T_+
\mathbf{Z}_+
+
\frac{\sigma_X^2}{\sigma_A^2}
\mathbf{I}_{K_+}
\right)^{-1}
\mathbf{Z}_+^T
$$


Reminder, equation 21 from the tutorial is the marginal probability of $\mathbf{X}$ given $\mathbf{Z}$, with $\mathbf{A}$ marginalized out:

$$
P(\mathbf{X} \mid \mathbf{Z}, \sigma_X, \sigma_A)
=
\frac
  {1}
  {(2\pi)^{ND/2} \sigma_X^{(N-K)D} \sigma_A^{KD} |\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2} \mathbf{I}|^{D/2} }
\exp \left(
  -\frac{1}{2\sigma_X^2}
  \mathrm{tr} \left(
    \mathbf{X}^T(\mathbf{I} - \mathbf{Z}(\mathbf{Z}^T\mathbf{Z} + \frac{\sigma_X^2}{\sigma_A^2} \mathbf{I})^{-1}\mathbf{Z}^T)\mathbf{X}
  \right)
\right)
$$

$$
=
\frac
  {1}
  {(2\pi)^{ND/2} \sigma_X^{(N-K_+)D} \sigma_A^{K_+D} |\mathbf{Z_+}^T\mathbf{Z_+} + \frac{\sigma_X^2}{\sigma_A^2} \mathbf{I}_{K_+}|^{D/2} }
\exp \left(
  -\frac{1}{2\sigma_X^2}
  \mathrm{tr} \left(
    \mathbf{X}^T(\mathbf{I} - \mathbf{Z}_+(\mathbf{Z}_+^T\mathbf{Z}_+ + \frac{\sigma_X^2}{\sigma_A^2} \mathbf{I}_{K+})^{-1}\mathbf{Z}_+^T)\mathbf{X}
  \right)
\right)
$$